In [1]:
import numpy as np
import json
import gzip
from scipy.sparse import coo_matrix
from scipy.stats import binned_statistic_2d
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pandas as pd
from collections import defaultdict

import torch
import torch.nn.functional as F
from numpy.linalg import eigvals
from torch_geometric.utils import (get_laplacian, to_scipy_sparse_matrix, to_undirected, to_dense_adj)
from torch_geometric.utils.num_nodes import maybe_num_nodes
# from torch_scatter import scatter_add

# Scipy eigendecomposition
from scipy.sparse.linalg import eigsh
from scipy.sparse.linalg import eigs

c:\Users\rebal\capstone-project\capstone\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/'
with gzip.open(data_dir + 'cells.json.gz', 'r') as f:
    cell_data = json.load(f)

widths = []
heights = []
for idx in range(len(cell_data)):
    width = cell_data[idx]['width']
    height = cell_data[idx]['height']
    widths.append(width)
    heights.append(height)

min_cell_width = np.min(widths)
max_cell_width = np.max(widths)
min_cell_height = np.min(heights)
max_cell_height = np.max(heights)

print('min cell width:', min_cell_width)
print('max cell width:', max_cell_width)
print('mean cell width:', np.mean(widths))
print('std cell width:', np.std(widths))
print()
print('min cell height:', min_cell_height)
print('max cell height:', max_cell_height)
print('mean cell height:', np.mean(heights))
print('std cell height:', np.std(heights))

widths = (widths - min_cell_width) / (max_cell_width - min_cell_width)
heights = (heights - min_cell_height) / (max_cell_height - min_cell_height)
print('Done processing cell sizes')

cell_to_edge_dict = {item['id']:{inner_item['id']: inner_item['dir'] for inner_item in item['terms']} for item in cell_data}
print('Done processing edge types dict')

min cell width: 256
max cell width: 397440
mean cell width: 14540.25
std cell width: 63888.21937522989

min cell height: 1536
max cell height: 503056
mean cell height: 16009.0
std cell height: 68982.92421027106
Done processing cell sizes
Done processing edge types dict


In [3]:
data_dir = '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/counter/1/'
with gzip.open(data_dir + 'counter.json.gz', 'r') as fin:
    instances_nets_data = json.load(fin)

instances = instances_nets_data['instances']
nets = instances_nets_data['nets']

inst_to_cell = {item['id']:item['cell'] for item in instances}

In [4]:
num_instances = len(instances)
num_nets = len(nets)

print('Number of instances:', num_instances)
print('Number of nets:', num_nets)

xloc_list = [instances[idx]['xloc'] for idx in range(num_instances)]
yloc_list = [instances[idx]['yloc'] for idx in range(num_instances)]
cell = [instances[idx]['cell'] for idx in range(num_instances)]
cell_width = [widths[cell[idx]] for idx in range(num_instances)]
cell_height = [heights[cell[idx]] for idx in range(num_instances)]
orient = [instances[idx]['orient'] for idx in range(num_instances)]

x_min = min(xloc_list)
x_max = max(xloc_list)
y_min = min(yloc_list)
y_max = max(yloc_list)

print('min xloc:', x_min)
print('max xloc:', x_max)
print('min yloc:', y_min)
print('max yloc:', y_max)

Number of instances: 23
Number of nets: 28
min xloc: 512
max xloc: 7552
min yloc: 4608
max yloc: 7680


In [5]:
X = np.expand_dims(np.array(xloc_list), axis = 1)
Y = np.expand_dims(np.array(yloc_list), axis = 1)
X = (X - x_min) / (x_max - x_min)
Y = (Y - y_min) / (y_max - y_min)

cell = np.expand_dims(np.array(cell), axis = 1)
cell_width = np.expand_dims(np.array(cell_width), axis = 1)
cell_height = np.expand_dims(np.array(cell_height), axis = 1)
orient = np.expand_dims(np.array(orient), axis = 1)

instance_features = np.concatenate((X, Y, cell, cell_width, cell_height, orient), axis = 1)

dictionary = {
    'num_instances': num_instances,
    'num_nets': num_nets,
    'x_min': x_min,
    'x_max': x_max,
    'y_min': y_min,
    'y_max': y_max,
    'min_cell_width': min_cell_width,
    'max_cell_width': max_cell_width,
    'min_cell_height': min_cell_height,
    'max_cell_height': max_cell_height,
    'instance_features': instance_features,
#     'sample_name': sample_name,
#     'folder': sample_names[0],
#     'design': design
}

In [6]:
connection_data = np.load(data_dir + 'counter_connectivity.npz')
dirs = []
edge_t = connection_data['data']
instance_idx = connection_data['row']

for idx in range(len(instance_idx)):
    inst = instance_idx[idx]
    cell = inst_to_cell[inst]
    edge_dict = cell_to_edge_dict[cell]
    t = edge_t[idx]
    direction = edge_dict[t]
    dirs.append(direction)

dirs = np.array(dirs)

In [7]:
connection_data['row']

array([ 0,  7,  9, 10, 13, 13,  6, 11,  5,  1,  6, 19,  1,  2,  3,  0,  2,
        8,  4, 18, 15, 14,  5, 21,  2, 22,  3, 14,  1, 12,  0,  1,  2,  3,
        0,  7,  6,  5,  6,  9,  4,  9,  8, 13, 11,  3,  9,  2,  7,  1, 10,
       12, 13, 16, 15, 20, 16, 18, 17,  3, 10,  8,  4, 20, 19, 21, 17, 22],
      dtype=int64)

In [8]:
connection_data['col']

array([ 0,  1,  1,  1,  1,  2,  3,  3,  3,  4,  5,  5,  6,  6,  6,  6,  7,
        7,  7,  7,  8,  8,  9,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 13,
       13, 14, 14, 15, 15, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20,
       21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 25, 25, 25, 26, 26, 27, 27],
      dtype=int64)

In [9]:
dirs

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1])

In [10]:
driver_sink_map = defaultdict(lambda: (None, []))

# Extract unique nodes and edges
nodes = list(set(connection_data['row']))
edges = list(set(connection_data['col']))

# Populate driver_sink_map
for node, edge, direction in zip(connection_data['row'], connection_data['col'], dirs):
    if direction == 1:  # Driver
        driver_sink_map[edge] = (node, driver_sink_map[edge][1])
    elif direction == 0:  # Sink
        driver_sink_map[edge][1].append(node)

# Convert to standard dictionary
driver_sink_map = dict(driver_sink_map)

In [11]:
driver_sink_map

{0: (None, [0]),
 1: (None, [7, 9, 10, 13]),
 2: (None, [13]),
 3: (5, [6, 11]),
 4: (None, [1]),
 5: (19, [6]),
 6: (None, [1, 2, 3, 0]),
 7: (18, [2, 8, 4]),
 8: (14, [15]),
 9: (2, [5, 21]),
 10: (3, [22]),
 11: (1, [14]),
 12: (12, [0]),
 13: (0, [1, 2, 3]),
 14: (6, [7]),
 15: (4, [5, 6, 9]),
 16: (8, [9]),
 17: (11, [13]),
 18: (9, [3]),
 19: (7, [2]),
 20: (10, [1]),
 21: (13, [12]),
 22: (15, [16]),
 23: (16, [20]),
 24: (17, [18]),
 25: (20, [3, 10, 8, 4]),
 26: (21, [19]),
 27: (22, [17])}

In [12]:
net_features = {}
for k, v in driver_sink_map.items():
    if v[0]:
        net_features[k] = [len(v[1]) + 1]
    else:
        net_features[k] = [len(v[1])]

In [13]:
net_features

{0: [1],
 1: [4],
 2: [1],
 3: [3],
 4: [1],
 5: [2],
 6: [4],
 7: [4],
 8: [2],
 9: [3],
 10: [2],
 11: [2],
 12: [2],
 13: [3],
 14: [2],
 15: [4],
 16: [2],
 17: [2],
 18: [2],
 19: [2],
 20: [2],
 21: [2],
 22: [2],
 23: [2],
 24: [2],
 25: [5],
 26: [2],
 27: [2]}

In [14]:
node_features = {}
for i in range(num_instances):
    node_features[i] = instance_features[i, 2:]
node_features

{0: array([2.30000000e+01, 4.83403158e-03, 0.00000000e+00, 0.00000000e+00]),
 1: array([6.50000000e+01, 9.02352562e-03, 0.00000000e+00, 0.00000000e+00]),
 2: array([6.50000000e+01, 9.02352562e-03, 0.00000000e+00, 6.00000000e+00]),
 3: array([6.50000000e+01, 9.02352562e-03, 0.00000000e+00, 6.00000000e+00]),
 4: array([5.80000000e+01, 1.28907509e-03, 0.00000000e+00, 6.00000000e+00]),
 5: array([4.80000000e+01, 6.44537544e-04, 0.00000000e+00, 6.00000000e+00]),
 6: array([7.00000000e+00, 1.28907509e-03, 0.00000000e+00, 6.00000000e+00]),
 7: array([4.80000000e+01, 6.44537544e-04, 0.00000000e+00, 2.00000000e+00]),
 8: array([4.20000000e+01, 6.44537544e-04, 0.00000000e+00, 4.00000000e+00]),
 9: array([7.00000000e+00, 1.28907509e-03, 0.00000000e+00, 4.00000000e+00]),
 10: array([4.80000000e+01, 6.44537544e-04, 0.00000000e+00, 2.00000000e+00]),
 11: array([3.40000000e+01, 3.22268772e-04, 0.00000000e+00, 4.00000000e+00]),
 12: array([3.40000000e+01, 3.22268772e-04, 0.00000000e+00, 0.00000000e+00

In [15]:
class Hypergraph:
    def __init__(self, nodes, edges, driver_sink_map):
        self.nodes = nodes
        self.edges = edges
        self.driver_sink_map = driver_sink_map

    def get_incident_edges(self, node):
        return [edge for edge in self.edges if node in self.driver_sink_map[edge][1] or node == self.driver_sink_map[edge][0]]

    def get_driver_and_sinks(self, edge):
        return self.driver_sink_map[edge]

In [16]:
hypergraph = Hypergraph(nodes, edges, driver_sink_map)

In [17]:
hypergraph.get_incident_edges(1)

[4, 6, 11, 13, 20]

In [18]:
congestion_fn = data_dir + 'counter_congestion.npz'
congestion_data = np.load(congestion_fn)
print('Congestion info:', congestion_fn)

congestion_data_demand = congestion_data['demand']
congestion_data_capacity = congestion_data['capacity']

num_layers = len(list(congestion_data['layerList']))
print('Number of layers:', num_layers)
print('Layers:', list(congestion_data['layerList']))

ybl = congestion_data['yBoundaryList']
xbl = congestion_data['xBoundaryList']

all_demand = []
all_capacity = []

for layer in list(congestion_data['layerList']):
    print('Layer', layer, ':')
    lyr = list(congestion_data['layerList']).index(layer)

    # Binned statistics 2D
    ret = binned_statistic_2d(xloc_list, yloc_list, None, 'count', bins = [xbl[1:], ybl[1:]], expand_binnumbers = True)

    i_list = np.array([ret.binnumber[0, idx] - 1 for idx in range(num_instances)])
    j_list = np.array([ret.binnumber[1, idx] - 1 for idx in range(num_instances)])

    # Get demand and capacity
    demand_list = congestion_data_demand[lyr, i_list, j_list].flatten()
    capacity_list = congestion_data_capacity[lyr, i_list, j_list].flatten()

    demand_list = np.array(demand_list)
    capacity_list = np.array(capacity_list)

    all_demand.append(np.expand_dims(demand_list, axis = 1))
    all_capacity.append(np.expand_dims(capacity_list, axis = 1))

    average_demand = np.mean(demand_list)
    average_capacity = np.mean(capacity_list)
    average_diff = np.mean(capacity_list - demand_list)
    count_congestions = np.sum(demand_list > capacity_list)

    print('    Number of demand > capacity:', count_congestions)
    print('    Average capacity - demand:', average_diff)
    print('    Average demand:', average_demand)
    print('    Average capacity:', average_capacity)

demand = np.concatenate(all_demand, axis = 1)
capacity = np.concatenate(all_capacity, axis = 1)

dictionary = {
    'demand': demand,
    'capacity': capacity
}

Congestion info: ../../data/chips/NCSU-DigIC-GraphData-2023-07-25/counter/1/counter_congestion.npz
Number of layers: 13
Layers: ['M1', 'MINT1', 'MINT2', 'MINT3', 'MINT4', 'MINT5', 'MSMG1', 'MSMG2', 'MSMG3', 'MSMG4', 'MSMG5', 'MG1', 'MG2']
Layer M1 :
    Number of demand > capacity: 0
    Average capacity - demand: 2.5217391304347827
    Average demand: 0.0
    Average capacity: 2.5217391304347827
Layer MINT1 :
    Number of demand > capacity: 0
    Average capacity - demand: 9.869565217391305
    Average demand: 0.9130434782608695
    Average capacity: 10.782608695652174
Layer MINT2 :
    Number of demand > capacity: 0
    Average capacity - demand: 12.391304347826088
    Average demand: 2.217391304347826
    Average capacity: 14.608695652173912
Layer MINT3 :
    Number of demand > capacity: 0
    Average capacity - demand: 11.565217391304348
    Average demand: 0.391304347826087
    Average capacity: 11.956521739130435
Layer MINT4 :
    Number of demand > capacity: 0
    Average capac

In [19]:
d = demand.sum(axis=1)
c = capacity.sum(axis=1)

In [20]:
cong = (c * 0.9) - d
out = (cong < 0).astype(int)
out

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [21]:
out = {}
for i in range(len(node_features)):
    out[i] = int(((c[i] * 0.9) - d[i]) < 0)

out

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0}

Node Features:

1. cell type (done)
2. cell width (done)
3. cell height (done)
4. orientation (done)
5. cell degree
6. 
7. 
8. persistence diagram
9. degree distribution of neighbors
10. 
11. 
12. 
13. 
14. 
15. 
16. 
17. 
18. eigenvector 1 (done)
19. eigenvector 2
20. eigenvector 3
21. eigenvector 4
22. eigenvector 5
23. eigenvector 6
24. eigenvector 7
25. eigenvector 8
26. eigenvector 9
27. eigenvector 10

Net Features:
1. Degree


In [32]:
coo = coo_matrix((connection_data['data'], (connection_data['row'], connection_data['col'])), shape=connection_data['shape'])
coo.toarray()

array([[1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 3, 0, 2, 0, 0, 0, 0, 6, 0, 5, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 3, 0, 6, 0, 0, 0, 5, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 6, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 3, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 2, 0, 0],
       [0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0],
       [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 4, 0, 0, 0,
        0, 0, 0, 

In [98]:
instance_idx = connection_data['row']
net_idx = connection_data['col']
num_instances = len(node_features)
num_nets = len(net_features)
net_idx += num_instances

In [ ]:
v1 = torch.unsqueeze(torch.Tensor(np.concatenate([instance_idx, net_idx], axis = 0)).long(), dim = 1)
v2 = torch.unsqueeze(torch.Tensor(np.concatenate([net_idx, instance_idx], axis = 0)).long(), dim = 1)
undir_edge_index = torch.transpose(torch.cat([v1, v2], dim = 1), 0, 1)

In [100]:
undir_edge_index

tensor([[ 0,  7,  9, 10, 13, 13,  6, 11,  5,  1,  6, 19,  1,  2,  3,  0,  2,  8,
          4, 18, 15, 14,  5, 21,  2, 22,  3, 14,  1, 12,  0,  1,  2,  3,  0,  7,
          6,  5,  6,  9,  4,  9,  8, 13, 11,  3,  9,  2,  7,  1, 10, 12, 13, 16,
         15, 20, 16, 18, 17,  3, 10,  8,  4, 20, 19, 21, 17, 22, 23, 24, 24, 24,
         24, 25, 26, 26, 26, 27, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31,
         32, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 36, 36, 37, 37, 38, 38, 38,
         38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47,
         47, 48, 48, 48, 48, 48, 49, 49, 50, 50],
        [23, 24, 24, 24, 24, 25, 26, 26, 26, 27, 28, 28, 29, 29, 29, 29, 30, 30,
         30, 30, 31, 31, 32, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 36, 36, 37,
         37, 38, 38, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45,
         45, 46, 46, 47, 47, 48, 48, 48, 48, 48, 49, 49, 50, 50,  0,  7,  9, 10,
         13, 13,  6, 11,  5,  1,  6, 19,  1,  2,  3,  0,  2

In [156]:
L = to_scipy_sparse_matrix(
    *get_laplacian(undir_edge_index, normalization = "sym", num_nodes = num_instances + num_nets)
)
evals, evects = eigsh(L, k = 10, which='SM')

In [157]:
evals

array([1.4975740e-07, 5.1986638e-02, 7.6044492e-02, 1.0287300e-01,
       1.4195484e-01, 1.6148876e-01, 2.0181537e-01, 2.6565453e-01,
       2.9000196e-01, 3.1038645e-01], dtype=float32)

In [158]:
evects

array([[ 1.71498626e-01,  8.89742374e-03,  1.86486226e-02,
        -2.65892595e-01, -2.10597426e-01, -8.39670748e-02,
        -2.55198598e-01,  3.67872939e-02,  2.55594134e-01,
        -1.66297201e-02],
       [ 1.91741243e-01, -1.01603977e-01,  1.39842369e-02,
        -1.60936385e-01, -3.13158602e-01, -1.24325454e-02,
         1.52744502e-01, -8.00782368e-02, -1.06439620e-01,
         1.56776801e-01],
       [ 1.91741258e-01,  4.83848006e-02,  1.72584243e-02,
        -1.39901042e-02, -1.64713919e-01,  6.27375841e-02,
        -3.16698775e-02,  1.03120103e-01,  1.37156695e-01,
        -3.59010696e-02],
       [ 1.91741213e-01, -3.27493995e-03,  1.20259047e-01,
        -7.29830116e-02, -1.20125771e-01,  1.04995415e-01,
        -5.26333153e-02, -2.38567621e-01, -1.42986402e-02,
         5.07383198e-02],
       [ 1.48522153e-01,  3.03359553e-02,  3.02814133e-02,
         5.39563186e-02,  1.90207623e-02,  1.90098017e-01,
         1.43464655e-02,  1.39325961e-01,  1.08263418e-02,
         7.